In [1]:
import pandas as pd

In [4]:
df_derechos_laborales = pd.read_csv(r'migracion_charly\regulacion_juridica_al_empleo.csv', sep=',', header=0)

Elimino columnas irrelevantes para el proyecto y/o redundantes

In [5]:
df_derechos_laborales.drop('source', axis=1, inplace=True)
df_derechos_laborales.drop('source.label', axis=1, inplace=True)
df_derechos_laborales.drop('indicator', axis=1, inplace=True)
df_derechos_laborales.drop('indicator.label',axis=1, inplace=True)

Cambio el nombre de las columnas

In [6]:
df_derechos_laborales.rename(columns={'ref_area':'codigo_pais', 'ref_area.label':'pais','time':'anio','obs_value':'derechos_laborales'}, inplace=True)

In [ ]:
remover = ['X01', 'X02', 'X03', 'X04', 'X05', 'X06', 'X07', 'X08', 'X09', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 
        'X21', 'X23', 'X24', 'X25', 'X26', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X38', 'X42', 'X44', 'X48', 'X49', 'X51', 
        'X52', 'X53', 'X54', 'X55', 'X56', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 
        'X73', 'X74', 'X75', 'X76', 'X77', 'X78', 'X79', 'X82', 'X83', 'X84', 'X86', 'X87', 'X89', 'X90', 'X91', 'X92', 'X94']

In [ ]:
indices = df_derechos_laborales[df_derechos_laborales["codigo_pais"].isin(remover)].index

In [ ]:
df_derechos_laborales.drop(indices, inplace=True)
df_derechos_laborales.reset_index(drop=True, inplace=True)

In [9]:
df_human_rights = pd.read_csv(r'migracion_charly\distribution-human-rights.csv', sep=',', header=0)

In [10]:
df_human_rights.rename(columns={'Entity':'pais', 'Code':'codigo_pais', 'Year':'anio', 'civ_libs_vdem_owid': 'derechos_humanos'}, inplace=True)

In [11]:
df_human_rights.dropna(subset=['derechos_humanos'], inplace=True)
df_human_rights.dropna(subset=['codigo_pais'], inplace=True)

In [13]:
df_human_rights = df_human_rights[(df_human_rights['anio'] >= 1990)]

In [ ]:
indices2 = df_human_rights[(df_human_rights['codigo_pais'] == 'OWID_GDR') | (df_human_rights['codigo_pais'] == 'OWID_WRL')].index

In [ ]:
df_human_rights.drop(indices2, inplace=True)
df_human_rights.drop('region', axis=1, inplace=True)

In [15]:
df_gender_wage_gap = pd.read_csv(r'migracion_charly\gender_wage_gap.csv', sep=',', header=0)

In [16]:
df_gender_wage_gap.drop('INDICATOR', axis=1, inplace=True)
df_gender_wage_gap.drop('MEASURE', axis=1, inplace=True)
df_gender_wage_gap.drop('FREQUENCY', axis=1, inplace=True)
df_gender_wage_gap.drop('Flag Codes', axis=1, inplace=True)

In [17]:
df_gender_wage_gap.rename(columns={'LOCATION':'codigo_pais', 'SUBJECT':'tipo', 'TIME':'anio', 'Value':'valor'}, inplace=True)

In [18]:
df_gender_wage_gap = df_gender_wage_gap[(df_gender_wage_gap['anio'] >= 1990)]

In [ ]:
indices3 = df_gender_wage_gap[(df_gender_wage_gap['codigo_pais'] == 'EU27') | (df_gender_wage_gap['codigo_pais'] == 'OECD')].index

In [ ]:
df_gender_wage_gap.drop(indices3, inplace=True)

In [19]:
df_gender_wage_gap_employee = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'EMPLOYEE'].rename(columns={'valor':'brecha_salarial_empleo'}).drop(['tipo'], axis=1)

In [20]:
df_gender_wage_gap_selfemployed = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'SELFEMPLOYED'].rename(columns={'valor':'brecha_salarial_autoempleo'}).drop(['tipo'], axis=1)

In [21]:
df_gender_wage_gap_merged = pd.merge(df_gender_wage_gap_employee, df_gender_wage_gap_selfemployed, on=['codigo_pais','anio'], how='outer')

In [24]:
df_gini = pd.read_csv(r'Etl\gini.csv')

In [26]:
df_gini.rename(columns={'Code':'codigo_pais', 'Entity':'pais', 'Year':'anio', 'Gini coefficient':'coeficiente_gini'}, inplace=True)

In [ ]:
df_gini = df_gini[(df_gini['anio'] >= 1990)]

In [ ]:
df_gini.loc[(df_gini['pais'] == 'Argentina - urban'),'codigo_pais'] = 'ARG'
df_gini.loc[(df_gini['pais'] == 'Bolivia - urban'),'codigo_pais'] = 'BOL'
df_gini.loc[(df_gini['pais'] == 'Uruguay - urban'),'codigo_pais'] = 'URY'
df_gini.loc[(df_gini['pais'] == 'Micronesia (country) - urban'),'codigo_pais'] = 'FSM'
df_gini.loc[(df_gini['pais'] == 'Suriname - urban'),'codigo_pais'] = 'SUR'
df_gini.loc[(df_gini['pais'] == 'Colombia - urban'),'codigo_pais'] = 'COL'
df_gini.loc[(df_gini['pais'] == 'Ecuador - urban'),'codigo_pais'] = 'ECU'

In [ ]:
indices4 = df_gini[(df_gini['pais'] == 'China - rural') | (df_gini['pais'] == 'Indonesia - urban') |
                   (df_gini['pais'] == 'China - urban') | (df_gini['pais'] == 'Indonesia - rural') |
                   (df_gini['pais'] == 'India - urban') | (df_gini['pais'] == 'India - rural')].index

In [ ]:
df_gini.drop(indices4, inplace=True)

In [27]:
merge1 = pd.merge(df_derechos_laborales, df_human_rights, on=['codigo_pais','anio'], how='outer')

In [ ]:
merge1['pais_x'].fillna(merge1['pais_y'], inplace=True)

In [ ]:
merge1.drop('pais_y', axis=1, inplace=True)

In [29]:
merge2 = pd.merge(merge1, df_gini, on=['codigo_pais','anio'], how='outer')

In [ ]:
merge2['pais_x'].fillna(merge2['pais'], inplace=True)

In [ ]:
merge2.drop('pais', axis=1, inplace=True)

In [30]:
fact_ind_sociopolitico = pd.merge(merge2, df_gender_wage_gap_merged, on=['codigo_pais',	'anio'], how='outer')

In [ ]:
cuenta = fact_ind_sociopolitico['codigo_pais'].value_counts()
codigos = cuenta[cuenta<10].index
indices5 = fact_ind_sociopolitico[fact_ind_sociopolitico['codigo_pais'].isin(codigos)].index
fact_ind_sociopolitico.drop(indices5, inplace=True)

In [ ]:
fact_ind_sociopolitico.sort_values(by=['codigo_pais', 'anio'], inplace=True)
fact_ind_sociopolitico.reset_index(drop=True, inplace=True)